## Preparação dos dados

* Carregue o dataset fornecido (simulado ou pequeno dataset público, ex:
Telco Customer Churn).
* Realize a limpeza de dados, tratando valores ausentes e inconsistências.
* Execute engenharia de features básicas (ex: one-hot encoding para variáveis
categóricas).

In [4]:
# packges 
import pandas as pd

# Replace with the actual filename and path
df = pd.read_csv("Telco-Customer-Churn.csv")

# Show first 5 rows
print(df.head())